# Project of Ana and Leo

In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from function import Utils
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas.api.types import is_numeric_dtype
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from PIL import Image 
import seaborn as sns

warnings.filterwarnings('ignore')

## Data Loading

In [2]:
df = pd.read_csv("hmw/Sampled_BPIC17_offer_log.csv")

# change according to used dataset:

input = {
    "Case ID" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Complete Timestamp": "ts",
    "(case) Accepted" : "y"  # change the outcome variable 
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

In [3]:
# # Countplot of the 'lifecycle:transition' column
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df, x='resource')
# plt.title('Count of resource')
# plt.xlabel('resource')
# plt.ylabel('Count')
# plt.show()


In [4]:
# def dt1(df):
#     df['delta_time'] = df.groupby('index')['ts'].shift(-1) - df['ts']
#     df['delta_time'] = df['delta_time'].fillna(pd.Timedelta(seconds=0)) # ? how to fill nan
#     return df['delta_time'].tolist()

# df["dt"] = dt1(df) 
# #df.drop("ts",axis=1,inplace = True)

# df.head()

## Data Preprocessing

In [5]:
# make delta time column from timestamps
df["dt"] = Utils.dt(df) # This needs to be improved
df.drop("ts", axis=1,inplace = True)

In [6]:
# show the percentage of nan values for each column of the dataset
Utils.persantage_nan(df)

index                            0.000000
caseid                           0.000000
activity                         0.000000
resource                         0.000000
Variant                          0.000000
Variant index                    0.000000
y                                0.000000
(case) ApplicationID             0.000000
(case) CreditScore               0.000000
(case) FirstWithdrawalAmount     0.000000
(case) MonthlyCost               0.000000
(case) NumberOfTerms             0.000000
(case) OfferedAmount             0.000000
(case) Selected                  0.000000
Action                           0.000000
EventID                          0.000000
EventOrigin                      0.000000
OfferID                         22.312464
lifecycle:transition             0.000000
dt                               0.000000
dtype: float64

In [7]:
# predict nan values for the column using RandomForest
df = Utils.predict_null_value(colum="OfferID",df=df) # apply if needed

In [8]:
# drop the columns that don't pass the threshold for nan vaues
df = Utils.prod_nan_with_treshold(df)
df

,index,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,...,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition,dt,OfferID_was_null
0,0,Offer_247135719,O_Create Offer,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,Created,Offer_247135719,Offer,Offer_247135719,complete,0 days 00:00:03.042000,True
1,1,Offer_247135719,O_Created,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete,0 days 00:02:12.568000,False
2,2,Offer_247135719,O_Sent (online only),User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
3,3,Offer_247135719,O_Cancelled,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
4,4,Offer_941964966,O_Create Offer,User_17,Variant 1,1,True,Application_196483749,0,4100.0,...,57,10000.0,False,Created,Offer_941964966,Offer,Offer_941964966,complete,0 days 00:00:01.551000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,Variant 2,2,True,Application_1574359134,767,6000.0,...,59,6000.0,True,Created,Offer_1064426652,Offer,Offer_520979602,complete,0 days 00:00:01.209000,True
22405,22405,Offer_1064426652,O_Created,User_23,Variant 2,2,True,Application_1574359134,767,6000.0,...,59,6000.0,True,statechange,OfferState_189916093,Offer,Offer_1064426652,complete,0 days 00:00:19.519000,False
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,Variant 2,2,True,Application_1574359134,767,6000.0,...,59,6000.0,True,statechange,OfferState_24747308,Offer,Offer_1064426652,complete,10 days 20:18:34.026000,False
22407,22407,Offer_1064426652,O_Returned,User_117,Variant 2,2,True,Application_1574359134,767,6000.0,...,59,6000.0,True,statechange,OfferState_1809485991,Offer,Offer_1064426652,complete,0 days 04:34:49.631000,False


In [9]:
# drop ID-columns:
drop_id_threshold = 0.5
df = Utils.drop_id_columns(df, drop_id_threshold)

Dropping column index with value: 1.0
Dropping column EventID with value: 1.0


In [10]:
df

,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventOrigin,OfferID,lifecycle:transition,dt,OfferID_was_null
0,Offer_247135719,O_Create Offer,User_17,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer,Offer_247135719,complete,0 days 00:00:03.042000,True
1,Offer_247135719,O_Created,User_17,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,Offer,Offer_247135719,complete,0 days 00:02:12.568000,False
2,Offer_247135719,O_Sent (online only),User_17,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
3,Offer_247135719,O_Cancelled,User_17,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
4,Offer_941964966,O_Create Offer,User_17,Variant 1,1,True,Application_196483749,0,4100.0,201.76,57,10000.0,False,Created,Offer,Offer_941964966,complete,0 days 00:00:01.551000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,Offer_1064426652,O_Create Offer,User_23,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,Created,Offer,Offer_520979602,complete,0 days 00:00:01.209000,True
22405,Offer_1064426652,O_Created,User_23,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,Offer,Offer_1064426652,complete,0 days 00:00:19.519000,False
22406,Offer_1064426652,O_Sent (mail and online),User_23,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,Offer,Offer_1064426652,complete,10 days 20:18:34.026000,False
22407,Offer_1064426652,O_Returned,User_117,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,Offer,Offer_1064426652,complete,0 days 04:34:49.631000,False


## Label Encoding

In [11]:
# group the data by caseid
df_grouped = df.groupby(["caseid"]).agg(list).reset_index()

In [12]:
df_grouped.head()

,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventOrigin,OfferID,lifecycle:transition,dt,OfferID_was_null
0,Offer_1000681710,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_20, User_20, User_20, User_117, User_115]","[Variant 2, Variant 2, Variant 2, Variant 2, V...","[2, 2, 2, 2, 2]","[True, True, True, True, True]","[Application_2131314372, Application_213131437...","[956, 956, 956, 956, 956]","[14500.0, 14500.0, 14500.0, 14500.0, 14500.0]","[200.0, 200.0, 200.0, 200.0, 200.0]","[96, 96, 96, 96, 96]","[16000.0, 16000.0, 16000.0, 16000.0, 16000.0]","[True, True, True, True, True]","[Created, statechange, statechange, statechang...","[Offer, Offer, Offer, Offer, Offer]","[Offer_520979602, Offer_1000681710, Offer_1000...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.663000, 0 days 00:00:19.05600...","[True, False, False, False, False]"
1,Offer_1001553250,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]","[Variant 2, Variant 2, Variant 2, Variant 2, V...","[2, 2, 2, 2, 2]","[False, False, False, False, False]","[Application_1607028451, Application_160702845...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]","[175.0, 175.0, 175.0, 175.0, 175.0]","[127, 127, 127, 127, 127]","[17500.0, 17500.0, 17500.0, 17500.0, 17500.0]","[True, True, True, True, True]","[Created, statechange, statechange, statechang...","[Offer, Offer, Offer, Offer, Offer]","[Offer_520979602, Offer_1001553250, Offer_1001...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.188000, 0 days 00:00:15.47200...","[True, False, False, False, False]"
2,Offer_1002136393,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]","[Variant 2, Variant 2, Variant 2, Variant 2, V...","[2, 2, 2, 2, 2]","[True, True, True, True, True]","[Application_638926349, Application_638926349,...","[969, 969, 969, 969, 969]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]","[343.25, 343.25, 343.25, 343.25, 343.25]","[20, 20, 20, 20, 20]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]","[True, True, True, True, True]","[Created, statechange, statechange, statechang...","[Offer, Offer, Offer, Offer, Offer]","[Offer_1456773456, Offer_1002136393, Offer_100...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.293000, 0 days 00:00:19.55600...","[True, False, False, False, False]"
3,Offer_1002236598,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_49, User_49, User_49, User_113, User_102]","[Variant 3, Variant 3, Variant 3, Variant 3, V...","[3, 3, 3, 3, 3]","[False, False, False, False, False]","[Application_818693455, Application_818693455,...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]","[210.0, 210.0, 210.0, 210.0, 210.0]","[127, 127, 127, 127, 127]","[21000.0, 21000.0, 21000.0, 21000.0, 21000.0]","[True, True, True, True, True]","[Created, statechange, statechange, statechang...","[Offer, Offer, Offer, Offer, Offer]","[Offer_1651128996, Offer_1002236598, Offer_100...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.269000, 0 days 00:00:11.34800...","[True, False, False, False, False]"
4,Offer_1002530118,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_15, User_15, User_15, User_1]","[Variant 1, Variant 1, Variant 1, Variant 1]","[1, 1, 1, 1]","[False, False, False, False]","[Application_1126987739, Application_112698773...","[0, 0, 0, 0]","[19750.0, 19750.0, 19750.0, 19750.0]","[210.12, 210.12, 210.12, 210.12]","[120, 120, 120, 120]","[20000.0, 20000.0, 20000.0, 20000.0]","[False, False, False, False]","[Created, statechange, statechange, statechange]","[Offer, Offer, Offer, Offer]","[Offer_1468908520, Offer_1002530118, Offer_100...","[complete, complete, complete, complete]","[0 days 00:00:01.216000, 0 days 0

In [13]:
df_grouped = Utils.reduce_list_columns(df_grouped)

In [14]:
# show the candidates for aggregation encoding
Utils.display_columns_to_aggragate(df_grouped)

,activity,resource,Action,OfferID,dt,OfferID_was_null
0,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_20, User_20, User_20, User_117, User_115]","[Created, statechange, statechange, statechang...","[Offer_520979602, Offer_1000681710, Offer_1000...","[0 days 00:00:01.663000, 0 days 00:00:19.05600...","[True, False, False, False, False]"
1,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]","[Created, statechange, statechange, statechang...","[Offer_520979602, Offer_1001553250, Offer_1001...","[0 days 00:00:01.188000, 0 days 00:00:15.47200...","[True, False, False, False, False]"
2,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]","[Created, statechange, statechange, statechang...","[Offer_1456773456, Offer_1002136393, Offer_100...","[0 days 00:00:01.293000, 0 days 00:00:19.55600...","[True, False, False, False, False]"
3,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_49, User_49, User_49, User_113, User_102]","[Created, statechange, statechange, statechang...","[Offer_1651128996, Offer_1002236598, Offer_100...","[0 days 00:00:01.269000, 0 days 00:00:11.34800...","[True, False, False, False, False]"
4,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_15, User_15, User_15, User_1]","[Created, statechange, statechange, statechange]","[Offer_1468908520, Offer_1002530118, Offer_100...","[0 days 00:00:01.216000, 0 days 00:00:20.01800...","[True, False, False, False]"
...,...,...,...,...,...,...
4995,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_73, User_73, User_73, User_43]","[Created, statechange, statechange, statechange]","[Offer_1468908520, Offer_993689039, Offer_9936...","[0 days 00:00:02.062000, 0 days 00:00:15.96300...","[True, False, False, False]"
4996,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_28, User_28, User_28, User_1]","[Created, statechange, statechange, statechange]","[Offer_1562802810, Offer_993800442, Offer_9938...","[0 days 00:00:01.530000, 0 days 00:00:20.28100...","[True, False, False, False]"
4997,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_19, User_19, User_19, User_116, User_113]","[Created, statechange, statechange, statechang...","[Offer_275638981, Offer_99473283, Offer_994732...","[0 days 00:00:01.283000, 0 days 00:04:24.72900...","[True, False, False, False, False]"
4998,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_25, User_25, User_25, User_119, User_102]","[Created, statechange, statechange, statechang...","[Offer_596592747, Offer_995784215, Offer_99578...","[0 days 00:00:02.593000, 0 days 00:00:15.82700...","[True, False, False, False, False]"


In [15]:
# some colomns have no importance in the process so we drop them
df_grouped = df_grouped.drop(['Variant'],axis=1) # domain knowledge: variant index is better for the model training and holds the same information
# df_grouped = df_grouped.drop(['Resource_failure_rate','Resource_Pass/Fail'],axis=1)

In [16]:
# perform index encoding on columns (domain knowledge)
df_grouped_indx = Utils.index_encoding(df_grouped, df, ["activity", "resource"])

In [17]:
# perform aggregation encoding on columns
df_grouped_agg = Utils.aggregation_encoding(df_grouped_indx,df)
df_grouped_agg

,caseid,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,...,Offer_435530126,Offer_144691980,Offer_1084451910,Offer_659154322,Offer_1064426652,True,False,avg_dt,max_dt,min_dt
0,Offer_1000681710,2,True,Application_2131314372,956,14500.0,200.00,96,16000.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 11:21:59.362200,7 days 04:11:16.866000,0 days 00:00:01.663000
1,Offer_1001553250,2,False,Application_1607028451,0,2500.0,175.00,127,17500.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,2 days 22:47:28.444400,6 days 13:57:17.486000,0 days 00:00:01.188000
2,Offer_1002136393,2,True,Application_638926349,969,6500.0,343.25,20,6500.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,5 days 22:58:29.630800,18 days 00:58:56.495000,0 days 00:00:01.293000
3,Offer_1002236598,3,False,Application_818693455,0,2500.0,210.00,127,21000.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 20:42:58.900800,3 days 03:26:35.948000,0 days 00:00:01.269000
4,Offer_1002530118,1,False,Application_1126987739,0,19750.0,210.12,120,20000.0,False,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,15 days 06:47:37.585000,30 days 13:35:04.553000,0 days 00:00:01.216000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Offer_993689039,1,True,Application_1786368292,0,1500.0,98.86,66,5500.0,False,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,0 days 12:14:52.968750,1 days 00:29:36.925000,0 days 00:00:02.062000
4996,Offer_993800442,1,True,Application_1082798467,0,13000.0,392.88,36,13000.0,False,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,15 days 10:42:01.801250,30 days 21:23:52.697000,0 days 00:00:01.530000
4997,Offer_99473283,2,False,Application_1183967721,817,5500.0,98.86,66,5500.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,6 days 03:10:21.290600,11 days 20:38:07.444000,0 days 00:00:01.283000
4998,Offer_995784215,2,True,Application_54468310,1051,6000.0,200.00,127,20000.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,2 days 15:08:14.918800,5 days 00:00:17.230000,0 days 00:00:02.593000


In [18]:
final_dataset = df_grouped_agg.copy()

In [19]:
# final check if the dataset is clean:

for col in final_dataset.columns:    
    x = final_dataset[col].iloc[0]

    if (col != "dt"):
        if(type(x) == list):
            print("Warning! Dataset not clean on column: " + col)

In [20]:
# final nan check
Utils.persantage_nan(final_dataset)

caseid                  0.0
Variant index           0.0
y                       0.0
(case) ApplicationID    0.0
(case) CreditScore      0.0
                       ... 
True                    0.0
False                   0.0
avg_dt                  0.0
max_dt                  0.0
min_dt                  0.0
Length: 5552, dtype: float64

In [21]:
final_dataset

,caseid,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,...,Offer_435530126,Offer_144691980,Offer_1084451910,Offer_659154322,Offer_1064426652,True,False,avg_dt,max_dt,min_dt
0,Offer_1000681710,2,True,Application_2131314372,956,14500.0,200.00,96,16000.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 11:21:59.362200,7 days 04:11:16.866000,0 days 00:00:01.663000
1,Offer_1001553250,2,False,Application_1607028451,0,2500.0,175.00,127,17500.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,2 days 22:47:28.444400,6 days 13:57:17.486000,0 days 00:00:01.188000
2,Offer_1002136393,2,True,Application_638926349,969,6500.0,343.25,20,6500.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,5 days 22:58:29.630800,18 days 00:58:56.495000,0 days 00:00:01.293000
3,Offer_1002236598,3,False,Application_818693455,0,2500.0,210.00,127,21000.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 20:42:58.900800,3 days 03:26:35.948000,0 days 00:00:01.269000
4,Offer_1002530118,1,False,Application_1126987739,0,19750.0,210.12,120,20000.0,False,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,15 days 06:47:37.585000,30 days 13:35:04.553000,0 days 00:00:01.216000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Offer_993689039,1,True,Application_1786368292,0,1500.0,98.86,66,5500.0,False,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,0 days 12:14:52.968750,1 days 00:29:36.925000,0 days 00:00:02.062000
4996,Offer_993800442,1,True,Application_1082798467,0,13000.0,392.88,36,13000.0,False,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,15 days 10:42:01.801250,30 days 21:23:52.697000,0 days 00:00:01.530000
4997,Offer_99473283,2,False,Application_1183967721,817,5500.0,98.86,66,5500.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,6 days 03:10:21.290600,11 days 20:38:07.444000,0 days 00:00:01.283000
4998,Offer_995784215,2,True,Application_54468310,1051,6000.0,200.00,127,20000.0,True,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,2 days 15:08:14.918800,5 days 00:00:17.230000,0 days 00:00:02.593000


## Model Selection

## Evaluation of Results